In [24]:
import string
import pandas as pd
import regex as re
import nltk
from unicodedata import normalize
from pickle import load
from pickle import dump
from collections import Counter

In [25]:
def load_doc(filename):

file = open(filename, mode='rt', encoding='utf-8')

text = file.read()

file.close()
return text

In [26]:
def to_sentences(doc):
return doc.strip().split('\n')

In [27]:
def sentence_lengths(sentences):
lengths = [len(s.split()) for s in sentences]
return min(lengths), max(lengths)

In [28]:
def load_doc(filename):

file = open(filename, mode='rt', encoding='utf-8')

text = file.read()

file.close()
return text


def to_sentences(doc):
return doc.strip().split('\n')


def sentence_lengths(sentences):
lengths = [len(s.split()) for s in sentences]
return min(lengths), max(lengths)

In [29]:
arabic_filename = 'data/Arab-Acquis/Arabic-Translations/test.en_ref.ar'
arabic_doc = load_doc(arabic_filename)
arabic_sentences = to_sentences(arabic_doc)
minlen1, maxlen1 = sentence_lengths(arabic_sentences)
print('Arabic data: sentences=%d, min=%d, max=%d' % (len(arabic_sentences), minlen1, maxlen1))


english_filename = 'data/Arab-Acquis/JRC-ACQUIS/ac-test.en'
english_doc = load_doc(english_filename)
english_sentences = to_sentences(english_doc)
minlen1, maxlen1 = sentence_lengths(english_sentences)
print('English data: sentences=%d, min=%d, max=%d' % (len(english_sentences), minlen1, maxlen1))

Arabic data: sentences=4107, min=1, max=246
English data: sentences=4107, min=1, max=269


In [30]:
arabic_sentences[0]

'مجلس الجماعة الاقتصادية الأوروبية'

In [31]:
english_sentences[0]

'THE COUNCIL OF THE EUROPEAN ECONOMIC COMMUNITY,'

In [32]:
import pandas as pd
df1 = pd.read_csv("data/arabic_english.txt",delimiter="\t",names=["eng","ar"])
df1

,eng,ar
0,Hi.,مرحبًا.
1,Run!,اركض!
2,Help!,النجدة!
3,Jump!,اقفز!
4,Stop!,قف!
...,...,...
24633,rising voices promoting a more linguistically ...,شاركنا تحدي ابداع ميم بلغتك الام تعزيزا للتنوع...
24634,following last year s successful campaign we i...,استكمالا لنجاح حملة العام السابق ندعوكم للمشار...
24635,during last year s challenge we also met langu...,تعرفنا خلال تحدي العام الماضي على ابطال لغويين...
24636,to take part just follow the simple steps outl...,للمشاركة في التحدي اتبع الخطوات الموضحة على ال...


In [33]:
import pandas as pd
df2 = pd.DataFrame(list(zip(english_sentences,arabic_sentences)), columns=['eng','ar'])
df2

,eng,ar
0,"THE COUNCIL OF THE EUROPEAN ECONOMIC COMMUNITY,",مجلس الجماعة الاقتصادية الأوروبية
1,Whereas the adoption of a common transport pol...,حيث أن اعتماد سياسة نقل مشتركة تنطوي من بين أم...
2,Article 1,المادة 1
3,3. The types of carriage listed in Annex II sh...,3. لا تخضع أنواع النقل المدرجة في الملحق الثان...
4,Member States shall inform the Commission of t...,تبلغ الدول الأعضاء المفوضية الأوروبية بالتدابي...
...,...,...
4102,Having regard to the request made by Luxembour...,باعتبار الطلب الذي تقدمت به لوكسمبورغ في 25 تم...
4103,"(2) Such derogations should be granted, at the...",(2) يجب أن يتم منح هذه الاستثناءات إلى النمسا ...
4104,Article 1,المادة 1
4105,(b) France is granted derogations for the prod...,(ب) تمنح فرنسا الاستثناءات للحصول على النتائج ...


In [34]:
data = pd.concat([df1, df2], ignore_index=True)
data

,eng,ar
0,Hi.,مرحبًا.
1,Run!,اركض!
2,Help!,النجدة!
3,Jump!,اقفز!
4,Stop!,قف!
...,...,...
28740,Having regard to the request made by Luxembour...,باعتبار الطلب الذي تقدمت به لوكسمبورغ في 25 تم...
28741,"(2) Such derogations should be granted, at the...",(2) يجب أن يتم منح هذه الاستثناءات إلى النمسا ...
28742,Article 1,المادة 1
28743,(b) France is granted derogations for the prod...,(ب) تمنح فرنسا الاستثناءات للحصول على النتائج ...


In [36]:
arabic_stopwords = set(nltk.corpus.stopwords.words("arabic"))
arabic_punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ'''

punctuations = arabic_punctuations + string.punctuation
def remove_stopwords(text):
    filtered_sentence = [w for w in text.split() if not w in punctuations]
    return ' '.join(filtered_sentence)

def clean_Data(line):
    if (isinstance(line, float)):
        return None
    line.replace('\n', ' ')
    line = ' '.join(line)
    translator = str.maketrans('', '', punctuations)
    line = line.translate(translator)
    line = ' '.join(line)
    return line

In [22]:
data.eng = data.eng.apply(clean_Data)

In [18]:
data

,eng,ar
0,H i,مرحبًا.
1,R u n,اركض!
2,H e l p,النجدة!
3,J u m p,اقفز!
4,S t o p,قف!
...,...,...
28740,H a v i n g r e g a r ...,باعتبار الطلب الذي تقدمت به لوكسمبورغ في 25 تم...
28741,2 S u c h d e r o ...,(2) يجب أن يتم منح هذه الاستثناءات إلى النمسا ...
28742,A r t i c l e 1,المادة 1
28743,b F r a n c e i s ...,(ب) تمنح فرنسا الاستثناءات للحصول على النتائج ...


In [21]:
data.to_csv('data/data.txt')

In [22]:
df = pd.read_csv("data/data.txt")
df = df.drop(df.columns[0], axis=1)
df

,eng,ar
0,Hi.,مرحبًا.
1,Run!,اركض!
2,Help!,النجدة!
3,Jump!,اقفز!
4,Stop!,قف!
...,...,...
28740,Having regard to the request made by Luxembour...,باعتبار الطلب الذي تقدمت به لوكسمبورغ في 25 تم...
28741,"(2) Such derogations should be granted, at the...",(2) يجب أن يتم منح هذه الاستثناءات إلى النمسا ...
28742,Article 1,المادة 1
28743,(b) France is granted derogations for the prod...,(ب) تمنح فرنسا الاستثناءات للحصول على النتائج ...
